In [1]:
import os 
from pathlib import Path
import logging

# Project - Setup

In [15]:
list_of_files = [
    "artifact/delete_later",

    "config/__init__.py",
    "config/config.yaml",
    "config/model_params.py",
    "config/path_config.py",

    "src/__init__.py",
    "src/data_ingestion.py",
    "src/data_preprocessing.py",
    "src/model_training.py",
    "src/logger.py",
    
    "utils/__init__.py",
    "utils/common_functions.py",


    "pipeline/__init__.py",
    "pipeline/traing_pipeline.py",
    
    "static/style.css",
    "templates/index.html",

    "application.py",

    "requirements.txt",
    "setup.py",

    "Dockerfile",
    "Jenkinsfile",
    "README.md",
]

In [16]:
for filepath in list_of_files:
    filepath = Path(filepath)
    filedir, filename = os.path.split(filepath)

    if filedir != "":
        os.makedirs(filedir, exist_ok=True)
        logging.info(f"Creating directory:{filedir} for the file {filename}")
    
    if (not os.path.exists(filepath)) or (os.path.getsize(filepath) == 0):
        with open(filepath,'w') as f:
            pass
            logging.info(f"Creating empty file: {filepath}")
    else:
        logging.info(f"{filename} is already exists")

# Requirements

In [10]:
%%writefile -a requirements.txt
pandas 
numpy
google-cloud-storage
scikit-learn
pyyaml
imbalanced-learn
lightgbm
mlflow
flask

Appending to requirements.txt


# Setup.py

In [ ]:
%%writefile -a setup.py
import setuptools

with open("README.md", "r", encoding="utf-8") as f:
    long_description = f.read()


__version__ = "0.0.0"

GITHUB_REPO_NAME = "Hotel-Reservation-Prediction"
GITHUB_AUTHOR_USER_NAME = "SunilKumar-ugra"
SRC_REPO = "src"
GITHUB_AUTHOR_EMAIL = "ugargolsunilkumar@gmail.com"



setuptools.setup(
    name=SRC_REPO,
    version=__version__,
    author=GITHUB_AUTHOR_USER_NAME,
    author_email=GITHUB_AUTHOR_EMAIL,
    description="Hotel-Reservation-Prediction",
    long_description=long_description,
    long_description_content="text/markdown",
    url=f"https://github.com/{GITHUB_AUTHOR_USER_NAME}/{GITHUB_REPO_NAME}",
    project_urls={
        "Bug Tracker": f"https://github.com/{GITHUB_AUTHOR_USER_NAME}/{GITHUB_REPO_NAME}/issues",
    },
    package_dir={"": "src"},
    packages=setuptools.find_packages(where="src")
)

In [13]:
%%writefile -a setup.py
from setuptools import setup,find_packages

with open("requirements.txt") as f:
    requirements = f.read().splitlines()

setup(
    name="Hotel-Reservation-Prediction",
    version="0.1",
    author="SunilKumar-ugra",
    packages=find_packages(),
    install_requires = requirements,
)

Appending to setup.py


In [ ]:
%%cmd 
pip install -e . 

Microsoft Windows [Version 10.0.19045.5487]
(c) Microsoft Corporation. All rights reserved.

(Hotel_reserv_pred) e:\Mlops_Udemy\Hotel Reservation Prediction>python setup.py


usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: setup.py --help [cmd1 cmd2 ...]
   or: setup.py --help-commands
   or: setup.py cmd --help

error: no commands supplied



(Hotel_reserv_pred) e:\Mlops_Udemy\Hotel Reservation Prediction>

# Custom Logging

In [ ]:
%%writefile -a src/logger.py
import logging
import os
from datetime import datetime
import sys

#LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"
LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y-%H')}.log" 
logs_path=os.path.join(os.getcwd(),"logs",LOG_FILE)
os.makedirs(logs_path,exist_ok=True)

LOG_FILE_PATH=os.path.join(logs_path,LOG_FILE) # log file path

logging.basicConfig(
    format="[%(asctime)s] - %(name)s - %(levelname)s - %(module)s file - LineNum:%(lineno)d - %(message)s",
    level=logging.INFO,
    handlers=[
        logging.FileHandler(LOG_FILE_PATH),     # file handler
        logging.StreamHandler(sys.stdout)       # console handler
    ]
)


logger=logging.getLogger("Hotel-Reservation-Prediction")

def get_logger(name):
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    return logger

Appending to src/logger.py


In [6]:
%%writefile -a src/logger.py
import logging
import os
from datetime import datetime

LOGS_DIR = "logs"
os.makedirs(LOGS_DIR,exist_ok=True)

LOG_FILE = os.path.join(LOGS_DIR, f"log_{datetime.now().strftime('%Y-%m-%d')}.log")

logging.basicConfig(
    filename=LOG_FILE,
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

def get_logger(name):
    logger = logging.getLogger(name)
    logger.setLevel(logging.INFO)
    return logger

Appending to src/logger.py


# Custom Exception

In [ ]:
import traceback
import sys

class CustomException(Exception):

    def __init__(self, error_message, error_detail:sys):
        super().__init__(error_message)
        self.error_message = self.get_detailed_error_message(error_message,error_detail)

    @staticmethod
    def get_detailed_error_message(error_message , error_detail:sys):

        _, _, exc_tb = traceback.sys.exc_info()
        file_name = exc_tb.tb_frame.f_code.co_filename
        line_number = exc_tb.tb_lineno

        return f"Error in {file_name} , line {line_number} : {error_message}"
    
    def __str__(self):
        return self.error_message
         



# Utils - commom Function

In [ ]:
%%writefile -a utils/common_functions.py
import os
import pandas
from src.logger import get_logger
from src.custom_exception import CustomException
import yaml
import pandas as pd

logger = get_logger(__name__)

def read_yaml(file_path):
    try:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File is not in the given path")
        
        with open(file_path,"r") as yaml_file:
            config = yaml.safe_load(yaml_file)
            logger.info("Succesfully read the YAML file")
            return config
    
    except Exception as e:
        logger.error("Error while reading YAML file")
        raise CustomException("Failed to read YAMl file" , e)
    

def load_data(path):
    try:
        logger.info("Loading data")
        return pd.read_csv(path)
    except Exception as e:
        logger.error(f"Error loading the data {e}")
        raise CustomException("Failed to load data" , e)
    

Overwriting utils/common_functions.py


In [2]:
from src.logger import logger

In [5]:
logger.info("Testing the logger")
logger.error("Testing the logger error")
logger.warning("Testing the logger warning")

[2025-03-12 16:38:07,361] - Hotel-Reservation-Prediction - INFO - 2012120710 file - LineNum:1 - Testing the logger
[2025-03-12 16:38:07,363] - Hotel-Reservation-Prediction - ERROR - 2012120710 file - LineNum:2 - Testing the logger error
[2025-03-12 16:38:07,365] - Hotel-Reservation-Prediction - WARNING - 2012120710 file - LineNum:3 - Testing the logger warning
